In [32]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [33]:
df = pd.read_csv('C:/Python/billing_data_dummied.csv', dtype={'Parent_ABA': str})

In [34]:
df.drop(columns='Unnamed: 0', inplace=True, axis=1)

In [56]:
data = df.pop('Total Volume')                         # Remove the volume column
df.insert(len(df.columns), 'Total Volume', data)     # Add it back in as the last column

In [62]:
# Filter for Morgan Stanley; for testing 026014630
df_MS =df[df['Parent_ABA'] == '026014630']

In [63]:
df_MS.drop(columns=['Parent_ABA', 'CYCLE_D'], inplace=True, axis=1)

C:\Users\G1RXK02\AppData\Local\Temp\ipykernel_23204\1982077826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_MS.drop(columns=['Parent_ABA', 'CYCLE_D'], inplace=True, axis=1)


In [64]:
# Normalize features
scaler = MinMaxScaler()

df_MS = scaler.fit_transform(df_MS)

In [71]:
y = df_MS[:, -1]        # Remove the volume column; set our target variable
x = df_MS[:, :-1]       # Filter out unnecessary columns for the model

In [72]:
from sklearn.model_selection import TimeSeriesSplit

# Use skleans TimeSeriesSplit to split our data for training
# Assumes the data is already sorted in ascending order by date
tss = TimeSeriesSplit(n_splits=3)

for train_index, test_index in tss.split(x):
    
    X_train, X_test = x[train_index, :], x[test_index, :]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
#X_train = pd.read_csv("C:/Python/billing_X_train.csv")
#X_test = pd.read_csv("C:/Python/billing_X_test.csv")

#y_train = pd.read_csv("C:/Python/billing_y_train.csv")
#y_test = pd.read_csv("C:/Python/billing_y_test.csv")

In [15]:
# Drop index column of pd dataframe
#X_train.drop('Unnamed: 0', inplace=True, axis=1)
#X_test.drop('Unnamed: 0', inplace=True, axis=1)
#y_train.drop('Unnamed: 0', inplace=True, axis=1)
#y_test.drop('Unnamed: 0', inplace=True, axis=1)

In [74]:
y_train

array([1.90063902e-02, 1.67286211e-07, 9.02525838e-02, ...,
       0.00000000e+00, 2.00743453e-06, 3.91516649e-03])

# LSTM

In [91]:
class LSTMPredictor(nn.Module):
    """
    LSTM-based time series predictor for multiple features.

    Args:
        num_features (int): Number of input features.
        n_hidden (int, optional): Number of hidden units in each LSTM layer. Default is 51.
        num_layers (int, optional): Number of LSTM layers. Default is 2.

    Attributes:
        num_features (int): Number of input features.
        n_hidden (int): Number of hidden units in each LSTM layer.
        num_layers (int): Number of LSTM layers.
        lstm_layers (nn.ModuleList): Stack of LSTM layers.
        linear (nn.Linear): Linear layer for prediction.

    Example:
        >>> model = LSTMPredictor(num_features=5, num_layers=3)
        >>> input_data = torch.randn(32, 10, 5)  # Batch size of 32, sequence length of 10
        >>> predictions = model(input_data)
    """

    def __init__(self, num_features, n_hidden=51, num_layers=2):
        """
        Initialize the LSTM predictor.

        Args:
            num_features (int): Number of input features.
            n_hidden (int, optional): Number of hidden units in each LSTM layer. Default is 51.
            num_layers (int, optional): Number of LSTM layers. Default is 2.
        """
        super(LSTMPredictor, self).__init__()
        self.num_features = num_features
        self.n_hidden = n_hidden
        self.num_layers = num_layers

        # Initialize a stack of LSTM layers
        self.lstm_layers = nn.ModuleList([
            nn.LSTMCell(num_features if i == 0 else n_hidden, n_hidden)
            for i in range(num_layers)
        ])

        # Linear layer for prediction
        self.linear = nn.Linear(n_hidden, 1)

    def forward(self, x, future=0):
        """
        Forward pass through the LSTM predictor.

        Args:
            x (torch.Tensor): Input data of shape (batch_size, seq_len, num_features).
            future (int, optional): Number of future time steps to predict. Default is 0.

        Returns:
            torch.Tensor: Predicted output of shape (batch_size, 1).
        """
        n_samples, seq_len, _ = x.size()

        # Initialize hidden states for each layer
        hidden_states = [
            (torch.zeros(n_samples, self.n_hidden, dtype=torch.float32),
            torch.zeros(n_samples, self.n_hidden, dtype=torch.float32))
            for _ in range(self.num_layers)
        ]

        # Iterate over each time step
        for t in range(seq_len):
            input_t = x[:, t, :]
            for i in range(self.num_layers):
                hidden_states[i] = self.lstm_layers[i](input_t, hidden_states[i])
                input_t = hidden_states[i][0]  # Pass output of previous layer as input

        # Use output of last layer
        output = self.linear(input_t)

        return output

In [94]:
class LSTMPredictor(nn.Module):
    """
    LSTM-based time series predictor for multiple features.

    Args:
        num_features (int): Number of input features.
        n_hidden (int, optional): Number of hidden units in each LSTM layer. Default is 51.
        num_layers (int, optional): Number of LSTM layers. Default is 2.

    Attributes:
        num_features (int): Number of input features.
        n_hidden (int): Number of hidden units in each LSTM layer.
        num_layers (int): Number of LSTM layers.
        lstm (nn.LSTM): LSTM layers.
        linear (nn.Linear): Linear layer for prediction.

    Example:
        >>> model = LSTMPredictor(num_features=5, num_layers=3)
        >>> input_data = torch.randn(32, 10, 5)  # Batch size of 32, sequence length of 10
        >>> predictions = model(input_data)
    """

    def __init__(self, num_features, n_hidden=51, num_layers=2):
        """
        Initialize the LSTM predictor.

        Args:
            num_features (int): Number of input features.
            n_hidden (int, optional): Number of hidden units in each LSTM layer. Default is 51.
            num_layers (int, optional): Number of LSTM layers. Default is 2.
        """

        super(LSTMPredictor, self).__init__()
        self.num_features = num_features
        self.n_hidden = n_hidden
        self.num_layers = num_layers

        # Define the LSTM layers
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=n_hidden, num_layers=num_layers, batch_first=True)

        # Define the output layer
        self.linear = nn.Linear(n_hidden, 1)

    def forward(self, x):
        """
        Forward pass through the LSTM predictor.

        Args:
            x (torch.Tensor): Input data of shape (batch_size, seq_len, num_features).

        Returns:
            torch.Tensor: Predicted output of shape (batch_size, 1).
        """
        
        # Pass the input through the LSTM layers
        # The LSTM layers return the outputs and the final hidden states
        lstm_out, _ = self.lstm(x)  

        # We only need the output at the last time step
        last_time_step = lstm_out[:, -1, :]  

        # Pass it through the output layer
        out = self.linear(last_time_step)  

        return out

In [76]:
# Convert to PyTorch tensors
X_train_tensors = torch.tensor(X_train, dtype=torch.float32)
X_test_tensors = torch.tensor(X_test, dtype=torch.float32)
y_train_tensors = torch.tensor(y_train, dtype=torch.float32)
y_test_tensors = torch.tensor(y_test, dtype=torch.float32)

In [105]:
num_features = len(X_train_tensors[0])
X_train_tensors[0] = X_train_tensors[0].view(-1, 1, num_features)
X_train_tensors[0].shape

torch.Size([94])

In [106]:
from ray import tune

def train_model(config):
    """
    Train an LSTM model with the given hyperparameters.

    Args:
        config (dict): Dictionary containing the hyperparameters for the model.

    Returns:
        dict: Dictionary containing the loss of the model.
    """
    # Define your model with the given hyperparameters
    model = LSTMPredictor(num_features=config["num_features"], n_hidden=config["n_hidden"], num_layers=config["num_layers"])

    # Define your loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

    # Train your model
    for epoch in range(100):  # You can adjust the number of epochs
        for i in range(len(X_train_tensors)):
            # Forward pass
            seq_len = 12  # replace with actual sequence length
            num_features = len(X_train_tensors[i]) // seq_len
            num_features = config["num_features"]
            X_train_tensors[i] = X_train_tensors[i].view(-1, seq_len, num_features)      # Ensure it's a 3D tensor
            outputs = model(X_train_tensors[i])
            loss = criterion(outputs, y_train_tensors[i])

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Report the loss to Tune, which is minimized
        tune.report(loss=loss.item())

# Define the hyperparameter search space
config = {
    "num_features": tune.choice([len(X_train_tensors[0])]),
    "n_hidden": tune.choice([10, 20, 30, 40, 50]),
    "num_layers": tune.choice([1, 2, 3]),
    "lr": tune.loguniform(1e-4, 1e-1),
}

# Run the hyperparameter tuning
analysis = tune.run(train_model, config=config, num_samples=10)

# Get the best hyperparameters
best_config = analysis.get_best_config(metric="loss", mode="min")
print(best_config)

2024-06-30 18:18:34,492	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2024-06-30 18:18:49,068	ERROR tune_controller.py:1331 -- Trial task failed for trial train_model_12828_00006
Traceback (most recent call last):
  File "C:\Users\G1RXK02\AppData\Roaming\Python\Python310\site-packages\ray\air\execution\_internal\event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "C:\Users\G1RXK02\AppData\Roaming\Python\Python310\site-packages\ray\_private\auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\G1RXK02\AppData\Roaming\Python\Python310\site-packages\ray\_private\client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\G1RXK02\AppData\Roaming\Python\Python310\site-packages\ray\_private\worker.py", line 2639, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
  File "C:\Users\G1RXK02\AppData\Roaming\Python\Python310\site-packages\ray\_private\worker.py", line 864, in get_objects
    raise value.as_instanceof_cau

Trial name
train_model_12828_00000
train_model_12828_00001
train_model_12828_00002
train_model_12828_00003
train_model_12828_00004
train_model_12828_00005
train_model_12828_00006
train_model_12828_00007
train_model_12828_00008
train_model_12828_00009


2024-06-30 18:18:49,081	ERROR tune_controller.py:1331 -- Trial task failed for trial train_model_12828_00005
Traceback (most recent call last):
  File "C:\Users\G1RXK02\AppData\Roaming\Python\Python310\site-packages\ray\air\execution\_internal\event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "C:\Users\G1RXK02\AppData\Roaming\Python\Python310\site-packages\ray\_private\auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\G1RXK02\AppData\Roaming\Python\Python310\site-packages\ray\_private\client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\G1RXK02\AppData\Roaming\Python\Python310\site-packages\ray\_private\worker.py", line 2639, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
  File "C:\Users\G1RXK02\AppData\Roaming\Python\Python310\site-packages\ray\_private\worker.py", line 864, in get_objects
    raise value.as_instanceof_cau

TuneError: ('Trials did not complete', [train_model_12828_00000, train_model_12828_00001, train_model_12828_00002, train_model_12828_00003, train_model_12828_00004, train_model_12828_00005, train_model_12828_00006, train_model_12828_00007, train_model_12828_00008, train_model_12828_00009])

In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import RandomizedSearchCV, KFold
from skorch import NeuralNetClassifier

# Wrap your model with skorch
model = NeuralNetClassifier(
    module=LSTMPredictor,
    max_epochs=100,  # Set an appropriate number of epochs
    lr=0.001,  # Initial learning rate (you can tune this too)
    optimizer=torch.optim.Adam,
    criterion = nn.MSELoss(),
    device='cuda' if torch.cuda.is_available() else 'cpu',
)

# Define hyperparameter search space
param_dist = {
    'module__n_hidden': [32, 64, 128],
    'module__num_features': [len(X_train[0])], 
    'module__num_layers': [2, 3, 4],  
    'lr': [0.001, 0.01, 0.1],  
    # Add other hyperparameters you want to tune
}

# Perform random grid search
random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=10,                          # Number of random samples
    #cv = KFold(n_splits=5, 
     #          shuffle=True, 
      #         random_state=42),        # Cross-validation folds
    scoring='neg_mean_squared_error',   # Choose an appropriate metric
    verbose=1,
    n_jobs=-1,                          # Use all available CPU cores
)

# Fit the random search to your data
random_search.fit(X_train, y = y_train)

# Get the best model and its hyperparameters
best_model = random_search.best_estimator_
best_params = random_search.best_params_

# Evaluate the best model on your validation set
val_accuracy = best_model.score(X_test, y_test)
print(f"Accuracy: {val_accuracy:.4f}")

In [ ]:
# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)  # Output layer

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Use the last time step's output
        return out

input_size = X_train.shape[1]
hidden_size = 64
num_layers = 2
model = LSTMModel(input_size, hidden_size, num_layers)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train.unsqueeze(1))
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate on test set
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test.unsqueeze(1))
    print(f"Test Loss: {test_loss.item():.4f}")

# Make predictions
predicted_volume = scaler.inverse_transform(test_outputs.numpy())

In [ ]:
import pandas as pd
import numpy as np

# Assuming you have the following arrays (replace with your actual data)
customer_ids = np.array([1, 2, 3, ...])  # Replace with your customer IDs
product_ids = np.array([101, 102, 103, ...])  # Replace with your product IDs

# Create a DataFrame for predicted volume
predicted_df = pd.DataFrame({
    'customer_id': customer_ids,
    'product_id': product_ids,
    'predicted_volume': predicted_volume.flatten()  # Flatten the 2D array
})

# Now you can join this DataFrame with your original data
# Assuming your original DataFrame is called 'original_df'
merged_df = original_df.merge(predicted_df, on=['customer_id', 'product_id'], how='left')

# 'merged_df' now contains the predicted volume alongside customer and product information
print(merged_df.head())  # Check the merged DataFrame

# LSTM 2

In [ ]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

class LSTM(nn.Module):
    """
    LSTM model for time series forecasting.

    Attributes:
        num_classes (int): The number of output classes.
        num_layers (int): The number of recurrent layers.
        input_size (int): The number of expected features in the input x.
        hidden_size (int): The number of features in the hidden state h.
        seq_length (int): The sequence length of the time series data.
    """
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        """
        Initialize the LSTM model.

        Args:
            num_classes (int): The number of output classes.
            num_layers (int): The number of recurrent layers.
            input_size (int): The number of expected features in the input x.
            hidden_size (int): The number of features in the hidden state h.
        """
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        """
        Forward pass through the LSTM layer.

        Args:
            x (torch.Tensor): The input to the LSTM layer.

        Returns:
            torch.Tensor: The output from the LSTM layer.
        """
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [ ]:
def train_model(config):
    """
    Train the LSTM model and report the loss to Tune.

    Args:
        config (dict): A dictionary containing the hyperparameters for training.
    """
    # Define model
    model = LSTM(input_size=1, hidden_size=int(config["hidden_size"]), output_size=1, num_layers=int(config["num_layers"]))
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

    # Train model
    epochs = 150
    for i in range(epochs):
        for seq, labels in zip(x_train_tensors, y_train_tensors):
            optimizer.zero_grad()
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                            torch.zeros(1, 1, model.hidden_layer_size))

            y_pred = model(seq)

            single_loss = loss_function(y_pred, labels)
            single_loss.backward()
            optimizer.step()

        # Report the loss to Tune
        with tune.checkpoint_dir(step=i) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=single_loss.item())

# Define the search space
config = {
    "lr": tune.loguniform(1e-4, 1e-1),
    "hidden_size": tune.choice([50, 100, 200]),
    "num_layers": tune.choice([1, 2, 3])
}

# Define the scheduler and reporter
scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=150,
    grace_period=1,
    reduction_factor=2)
reporter = CLIReporter(metric_columns=["loss", "training_iteration"])

In [ ]:
# Grab all the possible combinations of Parent ABA and Produce Code
combinations = df[['Parent_ABA', 'Service Code']].drop_duplicates()

# Define a dictionary to store the best model for each combination
best_models = {}

for _, row in combinations.iterrows():
    parent_aba = row['Parent_ABA']
    service_code = row['Service Code']

    # Filter data for this combination
    data = df[(df['Parent_ABA'] == parent_aba) & (df['Service Code'] == service_code]['Total Volume']
              
    # Preprocess data and split into training and testing sets
    # ... (insert your preprocessing and train-test split code here) ...

    # Convert your training and testing sets into PyTorch tensors
    # ... (insert your code to convert to PyTorch tensors here) ...

    # Run the hyperparameter search
    analysis = tune.run(train_model, config=config, scheduler=scheduler, progress_reporter=reporter)

    # Get the best model and store it in the dictionary
    best_trial = analysis.get_best_trial("loss", "min", "last")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")

    best_trained_model = LSTM(input_size=1, hidden_size=int(best_trial.config["hidden_size"]), output_size=1, num_layers=int(best_trial.config["num_layers"]))
    best_trained_model.load_state_dict(torch.load(os.path.join(best_trial.checkpoint.value, "checkpoint"))[0])

    best_models[(customer_id, product_id)] = best_trained_model

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate_model(models, x_test_tensors_dict, y_test_tensors_dict):
    """
    Evaluate the LSTM models on the test data.

    Args:
        models (dict): A dictionary containing the trained LSTM models.
        x_test_tensors_dict (dict): A dictionary containing the input sequences for the test data.
        y_test_tensors_dict (dict): A dictionary containing the target values for the test data.
    """
    
    for (customer_id, product_id), model in models.items():
        print(f"Evaluating model for customer_id {customer_id} and product_id {product_id}:")

        # Get the test data for this combination
        x_test_tensors = x_test_tensors_dict[(customer_id, product_id)]
        y_test_tensors = y_test_tensors_dict[(customer_id, product_id)]

        model.eval()  # Set the model to evaluation mode
        predictions = []
        with torch.no_grad():
            for seq in x_test_tensors:
                model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                                torch.zeros(1, 1, model.hidden_layer_size))
                predictions.append(model(seq).item())

        # Compare predictions to actual values
        for i in range(len(y_test_tensors)):
            print(f'Predicted: {predictions[i]}, Actual: {y_test_tensors[i].item()}')

        # Calculate and print the Mean Squared Error and Mean Absolute Error
        mse = mean_squared_error(y_test_tensors, predictions)
        mae = mean_absolute_error(y_test_tensors, predictions)
        print(f'Mean Squared Error: {mse}, Mean Absolute Error: {mae}')

In [ ]:
evaluate_model(best_models, x_test_tensors_dict, y_test_tensors_dict)

# XGBoost

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from scipy.stats import uniform, randint
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

In [ ]:
# Define your XGBoost model
xgb_model = XGBRegressor()

In [ ]:
# Specify hyperparameters and their potential distributions
param_dist = {
    'learning_rate': uniform(0.01, 0.2),
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 6),
    'subsample': uniform(0.8, 0.2),
    'colsample_bytree': uniform(0.8, 0.2),
    'reg_lambda': uniform(0.1, 10.0),
    'reg_alpha': uniform(0.1, 10.0)
}

# Perform randomized search
randomized_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=50, cv=5)
randomized_search.fit(X_train, y_train)

In [ ]:
# Get the best hyperparameters
best_params = randomized_search.best_params_
print("Best hyperparameters:", best_params

Tune hyper-parameters further based on best parameters results from the random grid search

In [ ]:
# Specify hyperparameters and their potential values
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'reg_lambda': [0.1, 1.0, 10.0],
    'reg_alpha': [0.1, 1.0, 10.0]
}

# Perform grid search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)